# Práctica 3

# Inteligencia Artificial 2022/23
## Belén Díaz Agudo -  Facultad de Informática UCM
## Búsqueda local
En esta primera parte usaremos ejercicios paso a paso para familiarizarnos con la resolución de problemas sencillos de optimización, problemas conocidos que vamos a resolver utilizando algoritmos de búsqueda local. 
En la segunda parte de la práctica se pide resolver el problema dado en el enunciado.

## Algoritmo de escalada
Hill Climbing es un algoritmo de búsqueda local heurística utilizada para problemas de optimización.
Esta solución puede o no ser el óptimo global. El algoritmo es una variante del algoritmo de generación y prueba.
<br>
En general, el algoritmo funciona de la siguiente manera:
- Evaluar el estado inicial.
- Si es igual al estado del objetivo, terminamos.
- Encuentra un estado vecino al estado actual
- Evaluar este estado. Si está más cerca del estado objetivo que antes, reemplace el estado inicial con este estado y repita estos pasos.
<br>
Usaremos la implementación de AIMA que está en el módulo search.py

```python
    def hill_climbing(problem):
        """From the initial node, keep choosing the neighbor with highest value,
        stopping when no neighbor is better. [Figure 4.2]"""
        current = Node(problem.initial)
        while True:
            neighbors = current.expand(problem)
            if not neighbors:
                break
            neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
            if problem.value(neighbor.state) <= problem.value(current.state):
                break
            current = neighbor
        return current.state

```

In [1]:
from search import *

In [2]:
initial = (0, 0)
grid = [[3, 7, 2, 8], [5, 2, 9, 1], [5, 3, 3, 1]]

In [3]:
# Pre-defined actions for PeakFindingProblem
directions4 = {'W': (-1, 0), 'N': (0, 1), 'E': (1, 0), 'S': (0, -1)}
directions8 = dict(directions4)
directions8.update({'NW': (-1, 1), 'NE': (1, 1), 'SE': (1, -1), 'SW': (-1, -1)})

class PeakFindingProblem(Problem):
    """Problem of finding the highest peak in a limited grid"""

    def __init__(self, initial, grid, defined_actions=directions4):
        """The grid is a 2 dimensional array/list whose state is specified by tuple of indices"""
        Problem.__init__(self, initial)
        self.grid = grid
        self.defined_actions = defined_actions
        self.n = len(grid)
        assert self.n > 0
        self.m = len(grid[0])
        assert self.m > 0

    def actions(self, state):
        """Returns the list of actions which are allowed to be taken from the given state"""
        allowed_actions = []
        for action in self.defined_actions:
            next_state = vector_add(state, self.defined_actions[action])
            if next_state[0] >= 0 and next_state[1] >= 0 and next_state[0] <= self.n - 1 and next_state[1] <= self.m - 1:
                allowed_actions.append(action)

        return allowed_actions

    def result(self, state, action):
        """Moves in the direction specified by action"""
        return vector_add(state, self.defined_actions[action])

    def value(self, state):
        """Value of a state is the value it is the index to"""
        x, y = state
        assert 0 <= x < self.n
        assert 0 <= y < self.m
        return self.grid[x][y]

In [4]:
problem = PeakFindingProblem(initial, grid, directions4)

In [5]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state

In [6]:
solution = problem.value(hill_climbing(problem))
solution

7

El resultado es `7`, porque parte del punto `(0, 0)`, cuyo valor de casilla es `3`. Desde allí, puede ir a derecha (7) y abajo(5). Elige ir a la derecha, y ahora tiene posibilidades de ir a izquierda (3), derecha (2) y abajo (2). Como todas son inferiores al valor actual, se para el problema en este **máximo local**.

# Enfriamiento simulado (simulated annealing) 
El algoritmo de enfriamiento simulado puede manejar las situaciones de óptimo local o mesetas típicas en algoritmos de escalada.
<br>
El enfriamiento simulado es bastante similar a la escalada pero en lugar de elegir el mejor movimiento en cada iteración, elige un movimiento aleatorio. Si este movimiento aleatorio nos acerca al óptimo global, será aceptado,
pero si no lo hace, el algoritmo puede aceptar o rechazar el movimiento en función de una probabilidad dictada por la temperatura.  Cuando la `temperatura` es alta, es más probable que el algoritmo acepte un movimiento aleatorio incluso si es malo. A bajas temperaturas, solo se aceptan buenos movimientos, con alguna excepción ocasional.
Esto permite la exploración del espacio de estado y evita que el algoritmo se atasque en el óptimo local.

Usaremos la implementación de AIMA del modulo search.py

```python
def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice
```

Como hemos visto en clase hay varios métodos de enfriamiento (scheduling routine) Se puede variar el método de enfriamiento. En la implementación actual estamos usando el método de enfriamiento exponencial (que se pasa como parámetro).
```python
def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)
```

In [7]:
simulated_annealing(problem)

(1, 0)

In [8]:
# Resolvemos el problema del máximo en una rejilla con enfriamiento simulado

solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
print(solutions)
max(solutions)

{1, 2, 3, 5, 7, 8, 9}


9

### Ejercicio 1.  Resuelve el problema anterior de encontrar el punto máximo en una rejilla. Comenta brevemente los resultados obtenidos en distintas rejillas con el algoritmo de escalada por máxima pendiente y enfriamiento simulado. 
Observa las características de la rejilla y la posición del máximo. 
Ejemplo de rejilla para pruebas en el que el máximo es 11.2, en la **posición (6, 6)** (lo usamos para calcular el % de éxitos)

In [9]:
grid = [[0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [1.00, 1.80, 1.90, 1.90, 1.80, 1.70, 1.60, 1.50, 0.00],
        [0.00, 1.90, 1.95, 0.40, 0.40, 0.00, 1.65, 0.00, 0.00],
        [0.00, 0.00, 0.00, 2.00, 0.00, 0.00, 2.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 3.00, 0.00, 3.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 7.00, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 7.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.25, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]

In [10]:
def evaluar_algoritmo(algoritmo,grid) -> int:
    veces_optimo = 0

    for i in range(12):
        for j in range(9):
            # Probar empezar desde cada una de las casillas de la matriz
            initial = (i, j)
            problem = PeakFindingProblem(initial, grid, directions8)
            #solution = problem.value(hill_climbing(problem))

            # Simular el problema 100 veces desde esta posición, al tener un factor de aleatoriedad, no son todas iguales
            # Y habrá intentos que alcanza mejores resultados
            solution = max(problem.value(algoritmo(problem)) for i in range(100))

            if solution == 11.2:
                # Se ha alcanzado el máximo en alguno de los 100 intentos
                veces_optimo += 1
    return veces_optimo

In [11]:
# Enfriamiento simulado
veces_optimo = evaluar_algoritmo(simulated_annealing,grid)

#for i in range(12):
#    for j in range(9):
#        # Probar empezar desde cada una de las casillas de la matriz
#        initial = (i, j)
#        problem = PeakFindingProblem(initial, grid, directions8)
#        #solution = problem.value(hill_climbing(problem))
#
#        # Simular el problema 100 veces desde esta posición, al tener un factor de aleatoriedad, no son todas iguales
#        # Y habrá intentos que alcanza mejores resultados
#        solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
#        solution = max(solutions)
#
#        if solution == 11.2:
#            # Se ha alcanzado el máximo en alguno de los 100 intentos
#            veces_optimo += 1

print(f"{veces_optimo * 100 / (9 * 12)}%")

89.81481481481481%


In [12]:
# Algoritmo de escalada
veces_optimo = evaluar_algoritmo(hill_climbing,grid)
#
#for i in range(12):
#    for j in range(9):
#        initial = (i, j)
#        problem = PeakFindingProblem(initial, grid, directions8)
#        #solution = problem.value(hill_climbing(problem))
#
#        # También prueba 100 veces el problema desde esta posición
#        # Pero el algoritmo de escalada no tiene factor de aleatoriedad,
#        #   entonces no tiene mucho sentido esto
#        # De hecho, si hacemos un print(solutions), sólo habrá un elemento en el conjunto de soluciones
#        solutions = {problem.value(hill_climbing(problem)) for i in range(100)}
#        solution = max(solutions)
#
#        if solution == 11.2:
#            veces_optimo += 1

print(f"{veces_optimo * 100 / (9 * 12)}%")

50.0%


Como el **método de enfriamiento tiene un factor de aleatoriedad**, hemos realizado varias ejecuciones, y se ha podido observar que la tasa de éxito ha sido siempre por encima de 85%. En comparación, el método de escalada, que es **determinista**, siempre obtiene una tasa de éxito de 50%. Por lo tanto, la diferencia es bastante considerable. 

La razón es porque en el método de escalada, se paraliza la ejecución al encontrarse con un máximo local. Mientras que en el de enfriamiento existe la posibilidad de ir a una casilla con valor inferior al del actual y seguir explorando. Eso reduce la probabilidad de estancarse en un máximo local.

El método de escalada ha podido encontrar la solución partiendo en ciertas casillas porque el máximo local más cercano es justo el máximo global.

Como consecuencia exploración, el **método de enfriamiento emplea un tiempo mayor de ejecución**. En las ejecuciones realizadas, el método de escalada emplea un tiempo de 1.5 segundos, mientras que el de enfriamiento, un tiempo superior a 35 segundos.

##  Algoritmos genéticos

Se define una clase ProblemaGenetico que incluye los elementos necesarios para la representación de un problema de optimización que se va a resolver con un algoritmo genético. Los elementos son los que hemos visto en clase:

 - `genes`: lista de genes usados en el genotipo de los estados.
 - `longitud_individuos`: longitud de los cromosomas
 - `decodifica`: función de obtiene el fenotipo a partir del genotipo.
 - `fitness`: función de valoración.
 - `muta`: función de mutación de un cromosoma 
 - `cruza`: función de cruce de un par de cromosomas

In [13]:
import random

In [14]:
class ProblemaGenetico(object):
    def __init__(self, genes, fun_dec, fun_muta, fun_cruza, fun_fitness, longitud_individuos):
        """Constructor de la clase"""
        self.genes = genes
        self.fun_dec = fun_dec
        self.fun_cruza = fun_cruza
        self.fun_muta = fun_muta
        self.fun_fitness = fun_fitness
        self.longitud_individuos = longitud_individuos

    
    def decodifica(self, genotipo):
        """Devuelve el fenotipo a partir del genotipo"""
        fenotipo = self.fun_dec(genotipo)
        return fenotipo

    def muta(self, cromosoma, prob):
        """Devuelve el cromosoma mutado"""
        mutante = self.fun_muta(cromosoma, prob)
        return mutante

    def cruza(self, cromosoma1, cromosoma2):
        """Devuelve el cruce de un par de cromosomas"""
        cruce = self.fun_cruza(cromosoma1, cromosoma2)
        return cruce

    def fitness(self, cromosoma):
        """Función de valoración"""
        valoracion = self.fun_fitness(cromosoma)
        return valoracion

**Problema a resolver: encontrar el valor X que optimice una función**

En primer lugar vamos a definir una instancia de la clase anterior correspondiente al problema de optimizar (maximizar o minimizar) 
Vamos a usar como ejemplo trivial la función cuadrado $x^2$ en el conjunto de los números naturales menores que $2^{10}$. 
Se usa este ejemplo trivial (del que sabemos la solución) para ver todos los elementos y poder observar el comportamiento del algoritmo genético.  Después deberás probar con otra función más compleja de tu elección. 

In [15]:
# Será necesaria la siguiente función que interpreta una lista de 0's y 1's como un número natural:  
# La siguiente función que interpreta una lista de 0's y 1's como
# un número natural:  

def binario_a_decimal(x):
    x = x[::-1]
    return sum(b * (2 ** i) for (i, b) in enumerate(x)) 

In [16]:
binario_a_decimal((1, 1, 1, 0))

14

In [17]:
# En primer luegar usaremos la clase anterior para representar el problema de optimizar (maximizar o minimizar)
# la función cuadrado en el conjunto de los números naturales menores que
# 2^{10}. 

# Vamos a definir funciones de cruce, mutación y fitness para este problema.

def fun_cruzar(cromosoma1, cromosoma2):
    """Cruza los cromosomas por la mitad (podemos cambiar la función de cruce eligiendo otro punto de cruce al azar)"""
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1 // 2] + cromosoma2[l1 // 2:l2]
    cruce2 = cromosoma2[0:l2 // 2] + cromosoma1[l2 // 2:l1]
    return [cruce1, cruce2]

def fun_mutar(cromosoma, prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0, l - 1)
    if prob > random.uniform(0, 1):
        # Invertir el bit [p]
        cromosoma[p] = (cromosoma[p] + 1) % 2
    return cromosoma


def fun_fitness_cuad(cromosoma):
    """Función de valoración que eleva al cuadrado el número recibido en binario"""
    n = binario_a_decimal(cromosoma) ** 2
    return n

class ProblemaCuadrados(ProblemaGenetico):
    def __init__(self,longitud_individuos,genes=[0,1]):
        super().__init__(genes,binario_a_decimal,fun_mutar,fun_cruzar,fun_fitness_cuad, longitud_individuos)
cuadrados = ProblemaCuadrados(genes=[0, 1], longitud_individuos=10)

Una vez definida la instancia cuadrados que representa el problema genético, probar alguna de las funciones definidas en la clase anterior, para esta instancia concreta. Por ejemplo:

In [18]:
cuadrados.decodifica([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1])
# Salida esperada: 1125

1125

In [19]:
cuadrados.fitness([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1])
# Salida esperada: 1265625

1265625

In [20]:
cuadrados.muta([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1], 0.1)
# Posible salida: [1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1]

In [21]:
cuadrados.muta([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1], 0.1)
# Posible salida: [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

In [22]:
cuadrados.cruza([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1], [0, 1, 1, 0, 1, 0, 0, 1, 1, 1])
# Posible salida: [[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

[[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

### Ejercicio 2

   - Os doy hecha una `función poblacion_inicial(problema_genetico,tamaño)`, para definir una población inicial de un tamaño dado, para una instancia dada de la clase anterior ProblemaGenetico

sugerencia: usar `random.choice`

   - Os doy hecha una función de cruce que recibe una instancia de `Problema_Genetico` y una población de padres (supondremos que hay un número par de padres), obtiene la población resultante de cruzarlos de dos en dos (en el orden en que aparecen)

`cruza_padres(problema_genetico,padres)`

   - Definir la función de mutación que recibe una instancia de `Problema_Genetico`, una población y una probabilidad de mutación, obtiene la población resultante de aplicar operaciones de mutación a cada individuo llamando a la función muta definida para el problema genético.

`muta_individuos(problema_genetico, poblacion, prob)`

In [23]:
def poblacion_inicial(problema_genetico, size):
    l = []  # población inicial
    for i in range(size):  # añadimos a la población size individuos
        x = []
        for j in range(problema_genetico.longitud_individuos):  # los individuos se generan eligiendo sus genes
            # de manera eleatoria de entre los genes posibles
            x.append(random.choice(problema_genetico.genes))
        l.append(x)
    return l

In [24]:
# Generar 10 números < 2^10
poblacion_inicial(cuadrados, 10)

[[0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
 [0, 1, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 1, 0, 0, 1],
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 0],
 [1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 1, 0, 1, 1, 1]]

In [25]:
def cruza_padres(problema_genetico, padres):
    l = []
    for i in range(len(padres) // 2):  # asumimos que la población de la que partimos tiene tamaño par
        desc = problema_genetico.fun_cruza(padres[2 * i],
                                           padres[2 * i + 1])  # El cruce se realiza con la función de cruce  
        # proporcionada por el propio problema genético
        l.append(desc[0])  # La población resultante se obtiene de cruzar el padre[0] con padre[1], padre[2] con padre[3]...
        l.append(desc[1])  # y añadir cada par de descendientes a la nueva población
    return l

In [26]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

cruza_padres(cuadrados, p1)
# Posible salida
# [[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
#  [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#  [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#  [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
#  [0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
#  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]]

[[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

In [27]:
def muta_individuos(problema_genetico, poblacion, prob):
    return [problema_genetico.muta(individuo,prob) for individuo in poblacion]

# hay que llamar a problema_genetico.muta(x,prob) para todos los individuos de la poblacion.

In [28]:
muta_individuos(cuadrados, p1, 0.5)
# Posible salida:
#  [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#   [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
#   [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
#   [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#   [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

[[1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 0, 1, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 0, 1]]

In [29]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

In [30]:
muta_individuos(cuadrados, p1, 0.5)

[[1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
 [0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

Vamos a definir una **función de selección** mediante torneo de `n` individuos de una población.

La función recibe como entrada:
 - una instancia de la clase `ProblemaGenetico`
 - una población
 - el número n de individuos que vamos a seleccionar
 - el número k de participantes en el torneo
 - un valor `opt` que puede ser o la función max o la función min (dependiendo de si el problema es de maximización o de minimización, resp.).

`seleccion_por_torneo(problema_genetico,poblacion,n,k,opt)`

Usar `random.sample` para seleccionar k elementos de una secuencia.

Por ejemplo, `random.sample(population=[2,5,7,8,9], k=3)` devuelve `[7,5,8]`. 

In [31]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion, k)

        # El parámetro key es para cambiar el criterio de ordenación
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

In [32]:
# Ejemplo
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados, 8), 3, 6, max)
# Posible salida: [[1, 1, 1, 1, 1, 0, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]]

[[1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 0, 0, 1, 0, 1, 0, 0]]

In [33]:
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados, 8), 3, 6, min)
# [[0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 1, 0, 1, 0]]

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 0, 1, 1, 1, 0, 0, 1, 0]]

In [34]:
# La siguiente función implementa una posibilidad para el algoritmo genético completo: 
# inicializa t = 0 
# Generar y evaluar la Población P(t)
# Mientras no hemos llegado al número de generaciones fijado:  t < nGen
#    P1 = Selección por torneo de (1-size)·p individuos de P(t)
#    P2 = Selección por torneo de (size·p) individuos de P(t)
#    Aplicar cruce en la población P2
#    P4 = Union de P1 y P3
#    P(t + 1) := Aplicar mutación P4 
#    Evalua la población P(t + 1) 
#    t:= t + 1

# Sus argumentos son:
# problema_genetico: una instancia de la clase ProblemaGenetico con la representación adecuada del problema de optimización 
# que se quiere resolver.
# k: número de participantes en los torneos de selección.
# opt: max ó min, dependiendo si el problema es de maximización o de minimización. 
# nGen: número de generaciones (que se usa como condición de terminación)
# size: número de individuos en cada generación
# prop_cruce: proporción del total de la población que serán padres. 
# prob_mutación: probabilidad de realizar una mutación de un gen.

def algoritmo_genetico(problema_genetico, k, opt, ngen, size, prob_cruces, prob_mutar):
    poblacion = poblacion_inicial(problema_genetico, size)
    n_padres = round(size * prob_cruces)
    n_padres = int(n_padres if n_padres % 2 == 0 else n_padres - 1)
    n_directos = size - n_padres
    for _ in range(ngen):
        poblacion = nueva_generacion(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar)

    mejor_cr = opt(poblacion, key=problema_genetico.fitness)
    mejor = problema_genetico.decodifica(mejor_cr)
    return (mejor, problema_genetico.fitness(mejor_cr)) 

Necesitarás definir la función auxiliar `nueva_generacion(problema_genetico, poblacion, n_padres, n_directos, prob_mutar)` que dada una población calcula la siguiente generación.

In [35]:
# Definir la función nueva_generacion
def nueva_generacion(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar):
    #Padres
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k, opt)
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres, k, opt)
    #Cruzamos a los padres entre ellos
    cruces = cruza_padres(problema_genetico, padres1)
    generacion = padres2 + cruces
    #Llevamos a cabo mutaciones si corresponde
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

### Ejercicio 3.  Ejecutar el algoritmo genético anterior, para resolver el problema anterior (tanto en minimización como en maximización) para la función cuadrado y para otra función. 

Se puede comenzar probando los resultados y comportamiento del algorimto para la función $x\mapsto x^2$ ya que sabemos la solución. 
Después hacer una valoración de resultados y comentarios sobre el comportamiento del algoritmmo para otra función. 
Puedes elegir cualquier otra función.
($f(x) = \left|\frac{x-12}{2+\sin(x)}\right|$  entre X=-5 y X=25)

En la resolución del problema hay que tener en cuenta que el algoritmo genético devuelve un par con el mejor fenotipo encontrado y su valoración.

Se puede calcular el número de veces que el algoritmo alcanza la solución óptima modificando parámetros como:

- `k`=número de participantes del torneo

- `ngen`=número de generaciones

- `size`=tamaño de la población

- `prob_cruce`=probabilidad de cruce

- `prob_mutacion`=probabilidad de mutacion

Realizar pruebas para un número suficiente de ejecuciones para las cuales queremos estudiar cual es el porcentaje de optimalidad del algoritmo para los distintos parámetros y de ahí sacar conclusiones. Para determinar el número de veces que una solución es éxitosa se pueden considerar soluciones aproximadas, por ejemplo, que para el problema de minimización es exitosa si la solución es menor que 23 y para el problema de maximización una solución será exitosa si es mayor que 1000.

#### Resolución de maximización y minimización de $x\mapsto x^2$ 

Procedemos a resolver, usando la función definida en el ejericicio anterior, el problema `cuadrados`, que consiste en maximizar (o minimizar, dependiendo del parámetro `opt`) la función $x^2$. El interés de usar esta función es que ya conocemos sus valores óptimos (sean máximos o mínimos) para cualquier intervalo. Así podemos comprobar fácilmente si las soluciones de `algoritmo_genetico` son o no aceptables. Recordamos que `algoritmo_genetico` devuelve una tupla con el punto donde se alcanza el valor óptimo (según el algoritmo), junto con el valor en sí.

Para poder hacernos una buena idea del rendimiento del algoritmo, y debido a que es un algoritmo no determinista, pedimos que resuelva el mismo problema varias veces, donde la población inicial se generará de forma aleatoria. A partir de estas ejecuciones podemos juzgar el rendimiento de `algoritmo_genético` tomando la media de las soluciones

In [36]:
def puntuacion_media_y_resultados(problema, iterations = 100, **kwargs):
    assert iterations > 0
    resultados = [algoritmo_genetico(problema, **kwargs) for _ in range(iterations)]
    print(f"Puntuación media: {sum(score for _, score in resultados)/iterations}")
    return resultados

Empezamos probando el algoritmo para resolver el problema de minimizar $x^2$ cuando los individuos pueden tener hasta 10 bits de tamaño. Sabemos que $x^2$ alcanza el mínimo en $0$, así que esperamos obtener $0$ como valor mínimo, y $0$ como individuo que alcanza ese valor.

In [37]:
resultados = puntuacion_media_y_resultados(ProblemaCuadrados(10), k=3, opt=min, ngen=20, size=10, prob_cruces=0.7, prob_mutar=0.1)

Puntuación media: 674.96


Observamos que el valor obtenido es notablemente más grande que el esperado. Pero no pasa nada, esto no significa que el algoritmo esté funcionando mal, simplemente que los números con los que estamos trabajando son muy grandes. Al fin y al cabo, si nuestros individuos tienen 10 bits para tomar valores, el valor máximo de función de fitness que podemos obtener es $1023^2 \approx 10^6$, el cual es bastante mayor que el resultado que obtenemos. (La función de fitness eleva al cuadrado el resultado de convertir binario al decimal.)

In [38]:
avg = sum(score for _,score in resultados) / len(resultados)
print(f"Relativamente, nuestros resultados se alejan un {avg / 1023**2}% del verdadero valor óptimo, lo cual es negligente")

Relativamente, nuestros resultados se alejan un 0.0006449510715899894% del verdadero valor óptimo, lo cual es negligente


Otra forma de ver que nuestro algoritmo se comporta adecuadamente es fijarse en las soluciones obtenidas. Todas estas comparten que tienen pocos `1`s en su representación, es decir, están compuestas mayoritariamente por `0`s. 

Esto tiene sentido, pues en nuestras funciones de cruce y mutación tratamos a todas las posiciones por igual, y estas benefician a la función de fitness cuando son 0. 

Esto resulta en que los `0`s sean más comunes en los individuos que han evolucionado. Además, aunque nunca expresamos nuestra preferencia por `1`s en las posiciones más bajas, los individuos también exhiben menos `1`s en posiciones altas que en bajas pues lo acaban "aprendiendo" de la función de fitness, que si realiza esa discriminación.

In [39]:
for individuo, _ in resultados:
    print(bin(individuo)[2:].rjust(10))

         0
         0
         0
         0
         0
         0
         0
    100000
         1
         1
  10000000
         0
         0
         0
         0
   1000000
         0
         0
       101
         0
         0
   1000000
         1
       101
         0
     10000
         0
    100000
        10
    100000
    100000
    100000
         0
         0
         0
         0
    100000
         0
         1
        10
         0
    100000
         0
         0
  10000000
         0
         0
         0
   1000000
         0
         0
         0
         0
         0
         0
    100000
       100
         0
         0
    100000
         0
         0
      1001
         0
         0
         0
      1000
         0
    100000
         0
      1011
         1
         0
    100000
         0
         0
         0
       101
         0
         0
        10
   1000000
         0
         0
         0
         0
       101
         0
       110
         1
   1000000

Así, pasamos a analizar nuestro algoritmo usando la siguiente función, que recoge las observaciones que hemos hecho

In [40]:
def analizar_algoritmo_genetico(problema: ProblemaGenetico, iteraciones = 100, opt = min, **kwargs):
    resultados = [algoritmo_genetico(problema,**kwargs,opt=opt) for _ in range(iteraciones)]
    
    puntuacion_media = sum(score for _, score in resultados)/iteraciones

    valor_maximo = (2**(problema.longitud_individuos)-1)**2
    valor_minimo = 0
    rango = valor_maximo - valor_minimo
    valor_optimo = opt(valor_maximo,valor_minimo)
    
    avg = sum(score for _, score in resultados) / iteraciones
    aprox_relativa = abs(valor_optimo - avg) / rango
    print(f"Puntuación media: {puntuacion_media}")
    print(f"Verdadero valor óptimo: {valor_optimo}")
    print(f"Aproximación relativa: {aprox_relativa}%")
    return puntuacion_media, valor_optimo, aprox_relativa

Ahora procedemos a probar el rendimiento del anterior algoritmo para distintos valores de los hiperparámetros

In [41]:
possible_k = [2,3,4]
possible_prob_cruces = [0.2, 0.7]
possible_prob_mutar = [0.1,0.5]
#metricas = {
#    "k": [],
#    "prob_cruces": [],
#    "prob_mutar": [],
#    "puntuacion_media": [],
#    "aproximacion_relativa": []
#}
for k in possible_k:
    for prob_cruces in possible_prob_cruces:
        for prob_mutar in possible_prob_mutar:
            print(f"=== {k=} {prob_cruces=} {prob_mutar=} ===")
            puntuacion_media, a, aproximacion_relativa = analizar_algoritmo_genetico(ProblemaCuadrados(10),
                                        iteraciones=1000,
                                        ngen=20,size=10, # Estos hiperparámetros los dejamso fijos, pues siempre ayuda incrementarlos
                                        k=k,opt=min,prob_cruces=prob_cruces,prob_mutar=prob_mutar)
            #metricas['k'].append(k)
            #metricas['prob_cruces'].append(prob_cruces)
            #metricas['prob_mutar'].append(prob_mutar)
            #metricas['puntuacion_media'].append(puntuacion_media)
            #metricas['aproximacion_relativa'].append(aproximacion_relativa)

            print()

=== k=2 prob_cruces=0.2 prob_mutar=0.1 ===
Puntuación media: 3958.308
Verdadero valor óptimo: 0
Aproximación relativa: 0.003782320413481136%

=== k=2 prob_cruces=0.2 prob_mutar=0.5 ===
Puntuación media: 8188.071
Verdadero valor óptimo: 0
Aproximación relativa: 0.007824026854487548%

=== k=2 prob_cruces=0.7 prob_mutar=0.1 ===
Puntuación media: 2819.21
Verdadero valor óptimo: 0
Aproximación relativa: 0.0026938670595845885%

=== k=2 prob_cruces=0.7 prob_mutar=0.5 ===
Puntuación media: 127.163
Verdadero valor óptimo: 0
Aproximación relativa: 0.00012150929405682977%

=== k=3 prob_cruces=0.2 prob_mutar=0.1 ===
Puntuación media: 2295.042
Verdadero valor óptimo: 0
Aproximación relativa: 0.002193003729471424%

=== k=3 prob_cruces=0.2 prob_mutar=0.5 ===
Puntuación media: 4101.188
Verdadero valor óptimo: 0
Aproximación relativa: 0.003918847924902225%

=== k=3 prob_cruces=0.7 prob_mutar=0.1 ===
Puntuación media: 1489.331
Verdadero valor óptimo: 0
Aproximación relativa: 0.00142311488740398%

=== k=

De los datos obtenidos, podemos ver que los mejores resultados se dan cuando tenemos `prob_cruce` alta, lo cual es esperado, y `prob_mutar` relativamente alta también. Esto también tiene sentido, pues aunque nuestra función de cruce introduce la posibilidad
de bajar el valor de cada individuo, en este problema de juguete la aleatoriedad es la única forma con la que "deshacerse" de los `1`s en los genes, lo cual supone un gran beneficio.

En el caso del problema de maximización, repetimos las mismas pruebas

In [42]:
for k in possible_k:
    for prob_cruces in possible_prob_cruces:
        for prob_mutar in possible_prob_mutar:
            print(f"=== {k=} {prob_cruces=} {prob_mutar=} ===")
            puntuacion_media, a, aproximacion_relativa = analizar_algoritmo_genetico(ProblemaCuadrados(10),
                                        iteraciones=1000,
                                        ngen=20,size=10, # Estos hiperparámetros los dejamso fijos, pues siempre ayuda incrementarlos
                                        k=k,opt=max,prob_cruces=prob_cruces,prob_mutar=prob_mutar)
            #metricas['k'].append(k)
            #metricas['prob_cruces'].append(prob_cruces)
            #metricas['prob_mutar'].append(prob_mutar)
            #metricas['puntuacion_media'].append(puntuacion_media)
            #metricas['aproximacion_relativa'].append(aproximacion_relativa)

            print()

=== k=2 prob_cruces=0.2 prob_mutar=0.1 ===
Puntuación media: 989127.503
Verdadero valor óptimo: 1046529
Aproximación relativa: 0.05484940885536853%

=== k=2 prob_cruces=0.2 prob_mutar=0.5 ===
Puntuación media: 934333.999
Verdadero valor óptimo: 1046529
Aproximación relativa: 0.10720677687861498%

=== k=2 prob_cruces=0.7 prob_mutar=0.1 ===
Puntuación media: 1003876.866
Verdadero valor óptimo: 1046529
Aproximación relativa: 0.04075580705360288%

=== k=2 prob_cruces=0.7 prob_mutar=0.5 ===
Puntuación media: 1037893.65
Verdadero valor óptimo: 1046529
Aproximación relativa: 0.008251419693099738%

=== k=3 prob_cruces=0.2 prob_mutar=0.1 ===
Puntuación media: 1007950.449
Verdadero valor óptimo: 1046529
Aproximación relativa: 0.0368633368019424%

=== k=3 prob_cruces=0.2 prob_mutar=0.5 ===
Puntuación media: 971417.698
Verdadero valor óptimo: 1046529
Aproximación relativa: 0.07177183049872486%

=== k=3 prob_cruces=0.7 prob_mutar=0.1 ===
Puntuación media: 1014265.903
Verdadero valor óptimo: 1046529

Los resultados obtenidos para el problema de maximización son casi equivalentes a los obtenidos para el problema de minimización, pero siempre con una aproximación relativa más grande. Esto tiene sentido, pues las diferencias entre el valor máximo y los valores obtenidos por el algoritmo no son uniformes, sino cuadráticas, así que cuanto más grandes sean los números, mayores serán las distancias relativas de las soluciones. Lo que se mantiene invariante es que los mejores resultados se dan con mayor `k`, y probabilidades más altas de cruce y de mutación.

#### Resolución de maximización y minimización de una función a elección del alumno

Para este apartado hemos elegido la **función Rosenbrock**, así que vamos a llevar a cabo el mismo proceso que hemos seguido para la función $x^2$. La función se define, para dos variables como:

$$
f(x,y) = (a-x)^2 + b (y - x^2)^2
$$

Vamos a empezar fijando una de las dos variables para que sea más sencillo resolverlo. Fijamos la variable $y$ a un valor. La función original presenta los mínimos en valores no enteros cercanos al cero. Es por eso que vamos a dividir la entrada de la función por 1000 para poder así representar números con tres cifras decimales como números enteros y de ésta forma obtener soluciones más aproximadas, sin perder la forma de representación en números binarios.

In [43]:
def rosenbrock(a,b,y,scale=1000):
    return lambda x : (a-x/scale)**2 + b*(y-(x/scale)**2)**2

In [44]:
print(rosenbrock(1,1,0)(1024))
print(rosenbrock(1,1,0)(4/7*1000))
print(rosenbrock(1,1,0)(0))

1.1000876277759999
0.29029571012078303
1.0


In [45]:
class RosenbrockProblem(ProblemaGenetico):
    def __init__(self,a=1,b=1,y=0):
        self.fun_fitness_rosenbrock = lambda xs : rosenbrock(a,b,y)(binario_a_decimal(xs))
        super().__init__([0,1], binario_a_decimal, self.fun_mutar, self.fun_cruzar, self.fun_fitness_rosenbrock, 10)
    
    @staticmethod
    def fun_cruzar(cromosoma1, cromosoma2):
        """Cruza los cromosomas en un punto aleatorio"""
        l1 = len(cromosoma1)
        l2 = len(cromosoma2)
        pto_medio = random.random()
        cruce1 = cromosoma1[0:int(l1*pto_medio)] + cromosoma2[int(l1*pto_medio):l2]
        cruce2 = cromosoma2[0:int(l2*pto_medio)] + cromosoma1[int(l2*pto_medio):l1]
        return [cruce1, cruce2]

    @staticmethod
    def fun_mutar(cromosoma, prob):
        """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
        l = len(cromosoma)
        p = random.randint(0, l - 1)
        if prob > random.uniform(0, 1):
            cromosoma[p] = (cromosoma[p] + 1) % 2
        return cromosoma


problema_rosenbrock = RosenbrockProblem(a = 1, b = 1, y = 0)

Procedemos a hacer una análisis similar al del problema anterior. La diferencia ahora es que no conocemos la solución del problema, así que nos limitaremos a devolver el valor encontrado por la función, y evaluar si es mejor o no que para otros parámetros

In [46]:
for k in possible_k:
    for prob_cruces in possible_prob_cruces:
        for prob_mutar in possible_prob_mutar:
            print(f"=== {k=} {prob_cruces=} {prob_mutar=} ===")
            resultados = [algoritmo_genetico(problema_rosenbrock,
                                        ngen=20,size=10,opt=min,
                                        k=k,prob_cruces=prob_cruces,prob_mutar=prob_mutar) 
                            for _ in range(100)]
            ind_optimo = sum(ind for ind, _ in resultados) / len(resultados)
            valor_optimo = sum(val for _, val in resultados) / len(resultados)
            print(f"Individuo óptimo: {ind_optimo}")
            print(f"Valor óptimo: {valor_optimo}")
            print()

=== k=2 prob_cruces=0.2 prob_mutar=0.1 ===
Individuo óptimo: 582.14
Valor óptimo: 0.29489088983823997

=== k=2 prob_cruces=0.2 prob_mutar=0.5 ===
Individuo óptimo: 584.4
Valor óptimo: 0.30121722807133994

=== k=2 prob_cruces=0.7 prob_mutar=0.1 ===
Individuo óptimo: 576.72
Valor óptimo: 0.29414616342982

=== k=2 prob_cruces=0.7 prob_mutar=0.5 ===
Individuo óptimo: 581.5
Valor óptimo: 0.29143585467828

=== k=3 prob_cruces=0.2 prob_mutar=0.1 ===
Individuo óptimo: 578.37
Valor óptimo: 0.2954990851428501

=== k=3 prob_cruces=0.2 prob_mutar=0.5 ===
Individuo óptimo: 575.51
Valor óptimo: 0.2955513727642899

=== k=3 prob_cruces=0.7 prob_mutar=0.1 ===
Individuo óptimo: 579.3
Valor óptimo: 0.29359304931108005

=== k=3 prob_cruces=0.7 prob_mutar=0.5 ===
Individuo óptimo: 581.12
Valor óptimo: 0.2917762435415998

=== k=4 prob_cruces=0.2 prob_mutar=0.1 ===
Individuo óptimo: 585.05
Valor óptimo: 0.29313467134756993

=== k=4 prob_cruces=0.2 prob_mutar=0.5 ===
Individuo óptimo: 587.04
Valor óptimo: 0.2

Comprobamos que los resultados obtenidos para distintas ejecuciones son muy similares, pero que como para el problema anterior, el algoritmo funciona mejor cuando tanto la probabilidad de cruce como la probabilidad de mutar son altas. Para un `k` fijo el peor rendimiento se da cuando es más probable mutar que cruzarse

### Ejercicio 4 (opcional)
Resolver mediante una configuración de un algoritmo genético el problema de los **Ocho Consecutivos** que distribuye los números 1 al 8 en las ocho casillas de la figura, con la condición de que no puede haber dos números consecutivos en casillas adyacentes. Se consideran casillas adyacentes aquéllas que comparten un lado o una esquina.

La siguiente configuración representa una solución al problema:

```
2 6 8 5 
7 4 1 4
```

Comenta el resultado y el rendimiento del algoritmo para distintos parámetros.   
    

Se trata de un **problema de secuenciación**: lo que importa es qué elementos aparecen junto a otros. Ya sabemos los elementos que debe de haber (números del 1 al 8), y lo que importa es su ordenación. Por lo tanto, a la hora de hacer las operaciones del algoritmo genético, tenemos que usar aquellos que sirven para las permutaciones.

Para la **codificación**, usamos una lista de 8 elementos, que contiene números del 1 al 8. Entonces no hace falta decodificar, ya que la propia lista es la representación.

No todas las ordenaciones son válidas. Como el tablero es relativamente pequeño, se puede comprobar de forma eficiente si una configuración es válida mediante un bucle.

En cuanto a la **función de fitness / idoneidad**, que ha de ser siempre positivo, una opción sencilla sería calcular el **número de casillas que no tienen conflicto**, que es sencillo de implementar, y penaliza también a las configuraciones inválidas dándoles un valor inferior.

------

Antes de empezar, implementamos operadores de mutación y cruce para permutaciones:

In [47]:
# Implementacion de los operadores de mutacion para permutaciones
def muta_insercion(cromosoma, prob):
    """Elegir dos alelos aleatoriamente y colocar el segundo justo después del primero"""
    mutante = cromosoma[:]
    if prob > random.uniform(0, 1):
        l = len(mutante)
        p1 = random.randint(0, l - 1)
        p2 = random.randint(p1, l - 1)
        # Poner el segundo justo después del primero
        mutante.insert(p1 + 1, mutante.pop(p2))

    return mutante

def muta_intercambio(cromosoma, prob):
    """Seleccionar dos alelos aleatoriamente e intercambiarlos"""
    mutante = cromosoma[:]
    if prob > random.uniform(0, 1):
        l = len(mutante)
        p1 = random.randint(0, l - 1)
        p2 = random.randint(p1, l - 1)
        # Intercambiar los dos alelos
        mutante[p1], mutante[p2] = mutante[p2], mutante[p1]

    return mutante

def muta_inversion(cromosoma, prob):
    """Seleccionar dos alelos aleatoriamente e invertir la cadena entre ellos"""
    mutante = cromosoma[:]
    if prob > random.uniform(0, 1):
        l = len(mutante)
        p1 = random.randint(0, l - 1)
        p2 = random.randint(p1, l - 1)
        # Invertir la cadena entre los dos alelos
        mutante[p1:p2] = reversed(mutante[p1:p2])

    return mutante

def muta_revuelto(cromosoma, prob):
    """Seleccionar un subconjunto de genes y reordenar aleatoriamente los alelos"""
    mutante = cromosoma[:]
    if prob > random.uniform(0, 1):
        l = len(mutante)
        p1 = random.randint(0, l - 1)
        p2 = random.randint(p1, l - 1)
        # Reordenar aleatoriamente los alelos entre p1 y p2
        mutante[p1:p2] = random.shuffle(mutante[p1:p2])

    return mutante

In [48]:
# Implementacion de los dos algoritmos de cruce para permutaciones

# Cruce de orden
def cruce_orden(cromosoma1, cromosoma2):
    """
    1. Seleccionar una parte arbitraria del primer padre
    2. Copiar esta parte en el hijo
    3. Copiar los números que no están en la primera parte
        Empezando desde el punto de cruce, y utilizando el orden en el que aparecen en el segundo padre
        Volviendo al principio del cromosoma cuando llega al final
    """

    l = len(cromosoma1) # Supongo que los dos cromosomas tienen la misma longitud

    # Una funcion auxliar para repetir el proceso
    # Porque el hijo2 se genera igual que hijo1, pero intercambiando los padres
    def copiar(padre1, padre2):
        hijo = [-1 for i in range(l)]

        p1 = random.randint(0, l - 1)
        p2 = random.randint(p1, l - 1)

        # Copiar la parte del primer padre entre p1 y p2
        hijo[p1:p2] = padre1[p1:p2]

        # Para los elementos que no están aún, copiarlos en el orden en el que aparecen en el segundo padre
        # Empezando desde p2 y volviendo al principio cuando llega al final
        i = p2  # Apunta al siguiente espacio libre en el hijo
        j = p2  # Apunta al siguiente elemento del padre2 que no se ha copiado
        pendiente = l - (p2 - p1)

        while pendiente > 0:
            if padre2[j] not in hijo:
                hijo[i] = padre2[j]
                i = (i + 1) % l
                pendiente -= 1
            j = (j + 1) % l

        return hijo

    cruce1 = copiar(cromosoma1, cromosoma2)
    cruce2 = copiar(cromosoma2, cromosoma1)
    
    return [cruce1, cruce2]

In [49]:
# Cruce PMX (Partially Mapped Crossover)
def cruce_pmx(cromosoma1, cromosoma2):
    """
    1. Elegir un segmento aleatorio y copiar desde P1
    2. Desde el PRIMER punto de cruce, buscar elementos de ese segmento en P2 que no se han copiado
    3. Para cada uno de esos elementos i, buscar que elementos j se han copiado en su lugar desde P1
    4. Colocar i en la posicion ocupada j de P2
    5. Si el lugar ocupado por j en P2 ya se ha rellenado en el hijo por elemento (k), poner i en la posicion ocupada por k en P2
    """

    l = len(cromosoma1) # Supongo que los dos cromosomas tienen la misma longitud

    def copiar(padre1, padre2):
        hijo = [-1 for i in range(l)]

        p1 = random.randint(0, l - 1)
        p2 = random.randint(p1, l - 1)

        # Copiar la parte del primer padre entre p1 y p2
        hijo[p1:p2] = padre1[p1:p2]

        # Indice para recorrer el p2
        # Apunta al siguiente elemento del padre2 que no se ha copiado en el hijo
        i = p1
        pendiente = l - (p2 - p1)   # Numero de elementos pendientes de copiar

        while pendiente > 0:
            if(padre2[i] not in hijo):
                # No está en el hijo, buscamos un hueco para él
                k = i

                while(hijo[k] != -1):
                    k = padre2.index(hijo[k])
                
                # Ahora k es la posicion donde debe de estar el elemento padre2[i] en el hijo
                hijo[k] = padre2[i]
                pendiente -= 1

            i = (i + 1) % l

        return hijo

    cruce1 = copiar(cromosoma1, cromosoma2)
    cruce2 = copiar(cromosoma2, cromosoma1)
    
    return [cruce1, cruce2]

-------------

Definimos el problema de Ocho Consecutivos como una clase, que hereda de `ProblemaGenetico`:

In [50]:
# Definimos una clase para todos los metodos que se usan en el algoritmo genetico para resolver el problema de Ocho Consecutivos
class OchoConsecutivos(ProblemaGenetico):
    def __init__(self, func_muta, func_cruza):
        """Constructor de la clase"""
        self.fun_cruza = func_cruza
        self.fun_muta = func_muta
        super().__init__([1, 2, 3, 4, 5, 6, 7, 8], self.fun_dec, func_muta, func_cruza, self.fun_fitness, 8)

    def fun_dec(self, genotipo):
        # Devolver un string representando el genotipo en tablero 2x4
        return " ".join(map(str, genotipo[0:4])) + " \n " + " ".join(map(str, genotipo[4:8]))

    @staticmethod
    def poblacion_inicial(size = 10):
        l = []
        
        for i in range(size): 
            # Generar una permutacion de numeros del 1 al 8
            x = [i for i in range(1, 9)]
            random.shuffle(x)
            l.append(x)

        return l

    def fun_fitness(self, cromosoma):
        """Calcula el numero de casillas que no tiene en su adyacencia un numero consecutivo"""
        result = 0

        for k in range(len(cromosoma)):
            i = k // 4
            j = k % 4

            valido = 1

            if i == 0 and abs(cromosoma[k] - cromosoma[(i + 1) * 4 + j]) == 1:
                # Casilla de abajo
                valido = 0

            if i == 1 and abs(cromosoma[k] - cromosoma[(i - 1) * 4 + j]) == 1:
                # Casilla de arriba
                valido = 0

            if j < 3 and abs(cromosoma[k] - cromosoma[i * 4 + j + 1]) == 1:
                # Casilla de la derecha
                valido = 0

            if j > 0 and abs(cromosoma[k] - cromosoma[i * 4 + j - 1]) == 1:
                # Casilla de la izquierda
                valido = 0

            result += valido

        return result

-------------

Vamos a probar si la clase funciona correctamente:

In [51]:
# Todas las casillas tienen un numero consecutivo en su adyacencia (fitness = 0)
OchoConsecutivos(muta_insercion, cruce_pmx).fitness([1, 2, 3, 4, 5, 6, 7, 8])

0

In [52]:
# Una solucion valida del problema (sale con fitness maximo 8)
OchoConsecutivos(muta_insercion, cruce_pmx).fitness([2, 6, 8, 5, 7, 4, 1, 3])

8

In [53]:
# Imprimir el tablero
print(OchoConsecutivos(muta_insercion, cruce_pmx).decodifica([2, 6, 8, 5, 7, 4, 1, 3]))

2 6 8 5 
 7 4 1 3


Probemos ahora resolver el problema con el algoritmo genético:

In [54]:
poblacion_inicial_size = 10
def algoritmo_genetico_ocho_consecutivos(poblacion_inicial = None, func_mut = muta_insercion, func_cruce = cruce_pmx, k = 2, ngen = 10, size = poblacion_inicial_size, prob_cruces = 0.6, prob_mutar = 0.1):
    problema = OchoConsecutivos(func_mut, func_cruce)
    poblacion = OchoConsecutivos.poblacion_inicial(size)

    if poblacion_inicial != None:
        poblacion = poblacion_inicial

    n_padres = round(size * prob_cruces)
    n_padres = int(n_padres if n_padres % 2 == 0 else n_padres - 1)
    n_directos = size - n_padres
    for _ in range(ngen):
        poblacion = nueva_generacion(problema, k, max, poblacion, n_padres, n_directos, prob_mutar)

    mejor_cr = max(poblacion, key=problema.fitness)
    mejor = problema.decodifica(mejor_cr)
    return (mejor, problema.fitness(mejor_cr)) 

Ahora procedemos a analizar el comportamiento del algoritmo genético para distintos parámetros. Principalmente, vamos a ver cómo afecta al variar:

- `k`: el número de participantes del torneo
- `nGen`: el número de generaciones
- La probabilidad de cruce
- La probabilidad de mutación

Cuando se realiza el análisis de uno de ellos, el resto de parámetros se ponen a su valor por defecto:

- `k = 2`
- `nGen = 10`
- `prob_cruce = 0.6`
- `prob_mutacion = 0.1`

Se utilizará la mutación por inserción y cruce pmx.

Para que el resultado sea más fácil de analizar, fijamos la población inicial y vemos qué ocurre.

In [55]:
poblacion_inicial_ocho_consecutivos = OchoConsecutivos.poblacion_inicial(poblacion_inicial_size)

In [56]:
def test_ocho_consecutivos(arg = None, iter = 1000):
    success = 0
    total_fitness = 0

    for i in range(iter):
        best, fitness = algoritmo_genetico_ocho_consecutivos(**arg)
        total_fitness += fitness
        if fitness == 8:
            success += 1
            # print("Solucion encontrada: " + str(best) + " con fitness " + str(fitness))

    # Imprimir tasa de exito
    print("Success: " + str(success) + "/" + str(iter))

    # Imprimir fitness medio
    print("Fitness medio: " + str(total_fitness / iter))

### Variar el número de participantes del torneo

Hemos probado a tomar los valores extremos de `k`, que selecciona el número de participantes en el torneo. En el caso de `k=1`, el algoritmo se comporta como un **algoritmo de búsqueda aleatoria**, ya que en cada torneo el mejor individuo es seleccionado al azar. En el caso de `k=10`, se comporta como una selección elitista, escogiendo aleatoriamente 10 y toma el mejor de ellos. 

Como consecuencia, **los individuos peores tienen menor probabilidad de ser seleccionados en `k=10`, y mayor probabilidad en `k=1`.**

Tras 1000 iteraciones, hemos observado que para el caso `k=1`, la tasa de éxito es bastante inferior en comparación con `k=10`.

In [57]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "k": 1})

Success: 193/1000
Fitness medio: 5.708


In [58]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "k": 10})

Success: 762/1000
Fitness medio: 7.524


### Variar el número de generaciones

Al variar el número de generaciones, el resultado variará menos o más de la población inicial. Al poner `ngen=0`, el resultado dependen totalmente de la población inicial, y no produce nuevas generaciones. Al poner `ngen=100`, el resultado es más variado, ya que el algoritmo ha tenido más tiempo para evolucionar.

In [59]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "ngen": 0})

Success: 0/1000
Fitness medio: 6.0


In [60]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "ngen": 20})

Success: 743/1000
Fitness medio: 7.486


Como consecuencia, el resultado de `ngen=0` es **determinista** una vez fijada la población inicial, mientras que el resultado de `ngen=100` es más aleatorio, y en general mejor al tener más tiempo para evolucionar. Esto se ve reflejado también en tiempo de ejecución, uno termina en un instante, mientras que el otro tarda más.

Si la población inicial es muy buena, el resultado de `ngen=0` puede superar al de `ngen=100`. Pero en general, el resultado de `ngen=100` es mejor.

**Para población inicial muy mala**, hemos podido observar que con `ngen=0` la tasa de éxito es 0.

### Variar la probabilidad de cruce

En el código se puede ver que, el parámetro `prob_cruce` es directamente propircional al `n_padres` que se seleccionan para el cruce. Y eso configura la cantidad de `padres1` que se seleccionan para el cruce. 

Cuando mayor sea la probabilidad de cruce, mayor será el número de individuos que se seleccionan para hacer el cruce. 

In [61]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "prob_cruces": 0})

Success: 226/1000
Fitness medio: 6.446


In [62]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "prob_cruces": 1})

Success: 677/1000
Fitness medio: 7.349


Si ponemos `prob_cruce=0`, no se selecciona ningún individuo para hacer el cruce, y el resultado de siguiente generación se obtiene por selección por torneo y mutación.

Hemos observado que la tasa de éxito con `prob_cruce=0` es inferior que con `prob_cruce=1`. Y el fitness medio es inferior también. Pensamos que es debido a que el **cruce introduce más diversidad en la población**, y consigue así explorar más el espacio de búsqueda, encontrando mejores soluciones. La mutación al tener una probabilidad baja (0.1) no es suficiente para explorar el espacio de búsqueda.

### Variar la probabilidad de mutación

In [63]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "prob_mutar": 0})

Success: 562/1000
Fitness medio: 7.117


In [64]:
test_ocho_consecutivos(arg={"poblacion_inicial": poblacion_inicial_ocho_consecutivos, "prob_mutar": 1})

Success: 625/1000
Fitness medio: 7.231


No se ha observado una diferencia significativa entre los resultados obtenidos con `prob_mutacion=0` y `prob_mutacion=1`, la tasa de éxito y el fitness medio son muy parecidos. (`prob_mutacion=0` es ligeramente peor.)

Pensamos que será porque el problema es pequeño, y con selección por torneo y cruce ya se puede obtener una buena solución.

### Ejercicio 5. 
Se os da el enunciado de forma separada. 

# Resolución del ejercicio 5

In [65]:
from search import *
import random

**Configuración del algoritmo genético**

1) *Representación*: vamos a representar el nanograma como una matriz de unos y ceros. Los unos representan que la casilla está llena y el cero que está vacía.
2) Para generar la *población inicial* vamos a colocar los cuadrados de manera aleatoria, pero vamos a **colocar en cada fila el número de cuadrados adecuado**.
3) La *función de evaluación/fitness* será la suma de condiciones que quedan por cumplir. Por cada condición que se incumpla sumamos 1, habremos encontrado una solución cuando su valor sea cero.
4) *Operadores genéticos*:
    - **Selección**: se hace de manera aleatoria, todos los padres dan lugar a descendencia.
    - **Cruce**: el cruce intercambia las filas de los padres para generar los hijos. Las posiciones de las filas permanecen fijas dentro del individuo, es decir, la fila 1 del padre siempre aparecerá como fila 1 de uno de los hijos y nunca como fila 2.
    - **Mutación**: la mutación intercambia la posición de casillas dentro de una misma fila, haciendo una "rotación" de los elementos de la misma. Se puede comparar con una simetría sobre un elemento de la fila.
5) *Parámetros del problema* (los iremos ajustando):
    - **Tamaño de la población inicial**: lo hemos fijado a 10, pero el código está diseñado para que se pueda modificar con facilidad.
    - **Probabilidad de cruce y de mutación (respectivamente)**: los hemos fijado a 0.7 y 0.1 porque son para los que hemos obtenido mejor resultado pero se puede modificar fácilmente.
    - **Condición de terminación**: cuando la función fitness de algún individuo de la población alcance el cero, es decir, que se cumplen todas. Se ofrecerá la posibilidad de seguir ejecutando el algoritmo aunque se haya alcanzado la solución, en caso de querer observar la evolución de la población.

Comenzamos definiendo las herramientas necesarias para resolver el problema mediante un algoritmo genético. Vamos a crear una subclase de problema genético para resolver este problema en concreto. 

Nuestro invariante a la hora de trabajar con el nanograma será que siempre vamos a tener cuidado de que en la fila siempre haya el número adecuado de cuadrados, es decir, la suma de las codiciones de la fila. Por ejemplo, si las condiciones de la fila 1 son [1,2,3] entonces en esa fila debemos colocar 6 cuadrados. Esto lo hemos tenido en cuenta a la hora de generar las poblaciones iniciales.

En la función fitness observamos cuantas condiciones(entendiendo como condiciones cada una de las sucesiones de cuadrados que se tiene que cumplir por cada fila, esto es, cada fila(y columna) siempre tiene más de una condición) se incumplen, y sumamos 1 por cada una de ellas.

In [66]:
#Las reglas del nanograma nos indican las restricciones de los cuadrados, es un vector de 2 * n, donde n es el tamaño del problema
# las primeras representas las restricciones de las filas y las siguientes las restricciones de las columnas
class Nanograma(ProblemaGenetico):
    def __init__(self, func_muta, func_cruza, reglas_nanograma, tam_nanograma):
        """Constructor de la clase"""
        super().__init__([0,1], self.fun_dec, func_muta, func_cruza, self.fun_fitness, tam_nanograma)
        self.fun_cruza = func_cruza
        self.fun_muta = func_muta
        self.reglas_nanograma = reglas_nanograma    #Lista de las reglas que debemos cumplir
        self.tam_nanograma = tam_nanograma          #Tamaño del problema(puede deducirse de las reglas, pero creemos que así 
                                                    # es más intuitivo)

    #Si la casilla está vacía se representa con ░░, en caso contrario utilizamos ██.
    def slick_show_problema(self, genotipo):
        res = "──" * self.tam_nanograma + '\n'
        for fila in genotipo:
            res += "".join('██' if vacio else '░░' for vacio in fila) + '\n'
        return res

    def fun_dec(self, genotipo):
        # Devolver un string representando el tablero del nanograma
        return self.slick_show_problema(genotipo)

    def poblacion_inicial(self, size = 10):
        poblacion = []

        #Primero sumamos las reglas de las filas para hallar el número de casillas llenas
        n_casillas_llenas = []
        for i in range(self.tam_nanograma):
            suma = 0
            for cond in self.reglas_nanograma[i]:
                suma += cond
            n_casillas_llenas.append(suma)
        

        #Ahora generamos las poblaciones en base a las sumas anteriores
        for i in range(size): 
            elem = []
            # Generar una matriz de 0 y 1 de acuerdo a lo explicado
            for j in range(self.tam_nanograma):
                fila = [1] * n_casillas_llenas[j] + [0] * (self.tam_nanograma - n_casillas_llenas[j])
                random.shuffle(fila)
                elem.append(fila) 
            poblacion.append(elem)

        return poblacion
    
    #Agrupa la lista de las condiciones de las filas
    def calcular_condiciones_fila(self, i, cromosoma):
        cont = 0
        cond = []
        for j in range(self.tam_nanograma):
            if cromosoma[i][j] == 1:
                cont = cont + 1
            else: 
                if cont != 0:
                    cond.append(cont)
                cont = 0
        if cont != 0:
            cond.append(cont)
        
        return cond
    
    #Agrupa las condiciones de las columna
    def calcular_condiciones_columna(self, i, cromosoma):
        cont = 0
        cond = []
        for j in range(self.tam_nanograma):
            if cromosoma[j][i] == 1:
                cont = cont + 1
            else: 
                if cont != 0:
                    cond.append(cont)
                cont = 0
        if cont != 0:
            cond.append(cont)
        
        return cond if len(cond) > 0 else [0]
                

    def fun_fitness(self, cromosoma):
        """Mira si no se cumplen las condiciones y en caso negativo suma 1"""
        result = 0
        #Miramos las condiciones de las filas 
        for i, expected in enumerate(self.reglas_nanograma):
            if i < self.tam_nanograma:
                actual = self.calcular_condiciones_fila(i, cromosoma)
            else:
                actual = self.calcular_condiciones_columna(i - self.tam_nanograma, cromosoma)
            result += (expected != actual)

        return result

Vamos a crear una función de cruce que intercambia las filas de los padres de manera que si tenemos los padres $P_1$ y $P_2$, sus hijos ($H_1$ y $H_2$) tengan las filas o bien de $P_1$ o bien de $P_2$, pero siempre manteniendo su posición relativa. De esta forma mantenemos el invariante de que el número de `1`s de una fila no cambia.

In [67]:
#Función de cruce específica del problema
def cruce_nanograma(cromosoma1, cromosoma2):
    #Generamos dos numeros aleatorios para hacer dos cortes
    tam_nanograma = len(cromosoma1)
    s1 = random.randrange(tam_nanograma)
    s2 = random.randrange(tam_nanograma)
    if s1 > s2:
        s1, s2 = s2, s1
    hijo_1 = cromosoma1[:s1] + cromosoma2[s1:s2] + cromosoma1[s2:]
    hijo_2 = cromosoma2[:s1] + cromosoma1[s1:s2] + cromosoma2[s2:]
    return hijo_1, hijo_2

**En el cruce no hay forma de que se varíen el contenido de las filas**, lo cual es un problema para la resolución del nonagrama. **Introducimos la posiblidad de que la fila de uno de los individuos varíe en la función de mutación**, que rota una fila aleatoria del individuo un offset aleatorio, con una probabilidad dada. De esta manera, mantenemos el invariante del número de `1`s, pero permitimos que nuevas filas sean introducidas a la población sin que estas existieran antes. Hemos probado con otras opciones para la función de mutación, como intercambiar casillas dentro de una misma fila, pero con la que hemos obtenido mejores resultados ha sido con la que presentamos a continuación.

In [68]:
#En este caso la función de mutación intercambia los valores sobre un pivote de una fila del cromosoma
def muta_ciclo(cromosoma, prob):
    mutacion = cromosoma[:]
    if random.uniform(0,1) < prob:
        i = random.randrange(len(mutacion))
        gen = cromosoma[i]
        j = random.randrange(1,len(gen))
        mutacion[i] = gen[j:] + gen[:j]
    return mutacion

Ahora hacemos un par de pruebas para asegurarnos de que las funciones definidas funcionan como deberían

In [69]:
nanograma_ejemplo = Nanograma(muta_ciclo, cruce_nanograma, [[1], [1], [2], [0]], 2)

Vamos a ver si la nueva clase funciona correctamente.

In [70]:
cromosoma_ejemplo = [[1, 0],[1, 0]]

In [71]:
nanograma_ejemplo.fitness(cromosoma_ejemplo)

0

In [72]:
cromosoma_ejemplo = [[1, 0],[0, 1]]

In [73]:
nanograma_ejemplo.fitness(cromosoma_ejemplo)

2

Como podemos observar, la función de evaluación funciona como debería. Veamos ahora que la representación:

In [74]:
#Imprimir el tablero
print(nanograma_ejemplo.decodifica(cromosoma_ejemplo))

────
██░░
░░██



In [75]:
#Generamos una población inicial para ver que funciona correctamente.
nanograma_ejemplo.poblacion_inicial()

[[[0, 1], [1, 0]],
 [[0, 1], [1, 0]],
 [[1, 0], [0, 1]],
 [[0, 1], [1, 0]],
 [[0, 1], [0, 1]],
 [[1, 0], [0, 1]],
 [[0, 1], [0, 1]],
 [[0, 1], [1, 0]],
 [[0, 1], [0, 1]],
 [[1, 0], [0, 1]]]

In [76]:
#Veamos que la fución de cruce funciona tal y como se ha descrito.
c1 = [[1,1,1],[1,1,1],[1,1,1]]
c2 = [[0]*3]*3
cruce_nanograma(c1,c2)

([[0, 0, 0], [1, 1, 1], [1, 1, 1]], [[1, 1, 1], [0, 0, 0], [0, 0, 0]])

In [77]:
#Veamos que la función de mutación cumple con lo descrito
cromosoma_a_mutar = [[0,0,1]]*3
muta_ciclo(cromosoma_a_mutar,1)

[[0, 0, 1], [0, 0, 1], [1, 0, 0]]

In [78]:
#Ahora toca ver si nuestra implementacion puede resolver el problema. Para ello ajustamos el algoritmo genético para nuestro
#problema.

def algoritmo_genetico_nanograma(
    func_mut = muta_ciclo,
    func_cruce = cruce_nanograma,
    reglas = [[1], [1], [2], [0]],
    tam_nanograma = 2,
    k = 2, # Parámetro de la selección por torneo
    ngen = 10,
    n_individuos = 10, 
    prob_cruces = 0.7,
    prob_mutar = 0.1,
    stop_at_best = True,
):
    problema = Nanograma(func_mut, func_cruce, reglas, tam_nanograma)
    poblacion = problema.poblacion_inicial(n_individuos)
    
    n_padres = round(n_individuos * prob_cruces)
    n_padres = int(n_padres - n_padres % 2)
    n_directos = n_individuos - n_padres

    print(f"""
Comienzo de la evolución con parámetros:
    {ngen=}
    {n_individuos=}
    func_mut={func_mut.__name__}
    func_cruce={func_cruce.__name__}
    {prob_cruces=}
    {prob_mutar=}
Inicio de la evolución:""")
    mejor_individuo_total = poblacion[1]
    mejor_puntuacion = problema.fitness(mejor_individuo_total)
    print(f"Generación {str(0).rjust(len(str(ngen)))}: Puntuación media {sum(problema.fitness(x) for x in poblacion) / len(poblacion)}")
    for gen in range(ngen):
        poblacion = nueva_generacion(problema, k, min, poblacion, n_padres, n_directos, prob_mutar)
        mejor_individuo = min(poblacion, key=problema.fitness)
        mejor_individuo_total = min((mejor_individuo_total,mejor_individuo), key=problema.fitness)
        mejor_puntuacion = problema.fitness(mejor_individuo_total)
        print(f"Generación {str(gen+1).rjust(len(str(ngen)))}: Puntuación media {sum(problema.fitness(x) for x in poblacion) / len(poblacion)}: Mejor puntuación de la generación: {mejor_puntuacion}")
        if stop_at_best and mejor_puntuacion == 0:
            return problema.fun_dec(mejor_individuo_total), mejor_puntuacion

    return problema.fun_dec(mejor_individuo_total), mejor_puntuacion

También reusamos la siguiente función para hacer análisis sobre varias ejecuciones.

In [79]:
def test_nanograma(iteraciones = 1,**kwargs):
    success = 0
    total_fitness = 0

    for i in range(iteraciones):
        best, fitness = algoritmo_genetico_nanograma(**kwargs)
        total_fitness += fitness
        if fitness == 0:
            success += 1
            print("Solucion encontrada:\n" + str(best) + " con fitness " + str(fitness))

    # Imprimir tasa de exito
    print("Success: " + str(success) + "/" + str(iteraciones))

    # Imprimir fitness medio
    print("Fitness medio de la mejor solucióna: " + str(total_fitness / iteraciones))

Probemos con una población de diez individuos y una sola iteración, con el resto de parámetros por defecto.

In [80]:
test_nanograma(n_individuos = 10, k = 1, iteraciones = 1)


Comienzo de la evolución con parámetros:
    ngen=10
    n_individuos=10
    func_mut=muta_ciclo
    func_cruce=cruce_nanograma
    prob_cruces=0.7
    prob_mutar=0.1
Inicio de la evolución:
Generación  0: Puntuación media 1.4
Generación  1: Puntuación media 1.8: Mejor puntuación de la generación: 0
Solucion encontrada:
────
██░░
██░░
 con fitness 0
Success: 1/1
Fitness medio de la mejor solucióna: 0.0


Puesto que el problema es pequeño, encontramos la solución rápidamente. Probemos a continuación con un problema más complejo, por ejemplo con el que aparece en el enunciado. Observamos que encontramos la solución en pocas generaciones, en cambio si disminuimos el número de hijos por generación no importa que demos lugar a muchas generaciones; el problema tarda mucho más en encontrar la solución(si llega a ella).

In [81]:
#Prueba con una población de 200 individuos.
test_nanograma(
    iteraciones=1,
    reglas = [[5], [2], [4], [2,1], [2,1]] + [[2,1],[5],[1,1,1],[1,1],[1,3]],
    tam_nanograma = 5,
    k = 3, # Parámetro de la selección por torneo
    ngen = 1000,
    n_individuos = 200, 
    prob_cruces = 0.7,
    prob_mutar = 0.1
)


Comienzo de la evolución con parámetros:
    ngen=1000
    n_individuos=200
    func_mut=muta_ciclo
    func_cruce=cruce_nanograma
    prob_cruces=0.7
    prob_mutar=0.1
Inicio de la evolución:
Generación    0: Puntuación media 6.995
Generación    1: Puntuación media 5.97: Mejor puntuación de la generación: 3
Generación    2: Puntuación media 5.105: Mejor puntuación de la generación: 0
Solucion encontrada:
──────────
██████████
████░░░░░░
░░████████
████░░░░██
░░████░░██
 con fitness 0
Success: 1/1
Fitness medio de la mejor solucióna: 0.0


In [106]:
# Prueba con una población de 10 individuos.
test_nanograma(
    iteraciones=1,
    reglas = [[5], [2], [4], [2,1], [2,1]] + [[2,1],[5],[1,1,1],[1,1],[1,3]],
    tam_nanograma = 5,
    k = 3, # Parámetro de la selección por torneo
    ngen = 10000,
    n_individuos = 80, 
    prob_cruces = 0.7,
    prob_mutar = 0.1
)


Comienzo de la evolución con parámetros:
    ngen=10000
    n_individuos=80
    func_mut=muta_ciclo
    func_cruce=cruce_nanograma
    prob_cruces=0.7
    prob_mutar=0.1
Inicio de la evolución:
Generación     0: Puntuación media 6.9875
Generación     1: Puntuación media 5.725: Mejor puntuación de la generación: 3
Generación     2: Puntuación media 5.0: Mejor puntuación de la generación: 3
Generación     3: Puntuación media 4.325: Mejor puntuación de la generación: 0
Solucion encontrada:
──────────
██████████
████░░░░░░
░░████████
████░░░░██
░░████░░██
 con fitness 0
Success: 1/1
Fitness medio de la mejor solucióna: 0.0


Observamos por las distintas pruebas que cuando el número de hijos es bajo (entre 10 y 100) por mucho que aumentemos el número de generaciones, el algoritmo se acerca a la solución pero no suele llegar a ancanzarla. Por otro lado, si aumentamos el número de individuos de la población (entre 150 y 200) en menos de diez generaciones conseguirmos alcanzar las soluciones. Esto tiene sentido, pues en algoritmos genéticos es más importante la variedad genética que el número de iteraciones, y cuantos más individuos halla, mayor diversidad genética acaba habiendo.

In [82]:
#Vamos con un ejemplo más complejo
test_nanograma(
    iteraciones=1,
    reglas = [[1], [2], [1,2], [1,1,2,2], [2,4], [1,2,1], [1,2]] + [[3],[2],[3,2],[2,4],[1,2,1],[2,2],[1]],
    tam_nanograma = 7,
    n_individuos = 300,
    k = 3, # Parámetro de la selección por torneo
    ngen = 1000,
    prob_cruces = 0.7,
    prob_mutar = 0.1
)


Comienzo de la evolución con parámetros:
    ngen=1000
    n_individuos=300
    func_mut=muta_ciclo
    func_cruce=cruce_nanograma
    prob_cruces=0.7
    prob_mutar=0.1
Inicio de la evolución:
Generación    0: Puntuación media 11.346666666666666
Generación    1: Puntuación media 10.41: Mejor puntuación de la generación: 7
Generación    2: Puntuación media 9.646666666666667: Mejor puntuación de la generación: 7
Generación    3: Puntuación media 8.926666666666666: Mejor puntuación de la generación: 6
Generación    4: Puntuación media 8.32: Mejor puntuación de la generación: 6
Generación    5: Puntuación media 7.633333333333334: Mejor puntuación de la generación: 6
Generación    6: Puntuación media 7.066666666666666: Mejor puntuación de la generación: 5
Generación    7: Puntuación media 6.51: Mejor puntuación de la generación: 5
Generación    8: Puntuación media 6.073333333333333: Mejor puntuación de la generación: 5
Generación    9: Puntuación media 5.79: Mejor puntuación de la generac

Generación   94: Puntuación media 4.25: Mejor puntuación de la generación: 4
Generación   95: Puntuación media 4.34: Mejor puntuación de la generación: 4
Generación   96: Puntuación media 4.256666666666667: Mejor puntuación de la generación: 4
Generación   97: Puntuación media 4.203333333333333: Mejor puntuación de la generación: 4
Generación   98: Puntuación media 4.1433333333333335: Mejor puntuación de la generación: 4
Generación   99: Puntuación media 4.246666666666667: Mejor puntuación de la generación: 4
Generación  100: Puntuación media 4.243333333333333: Mejor puntuación de la generación: 4
Generación  101: Puntuación media 4.303333333333334: Mejor puntuación de la generación: 4
Generación  102: Puntuación media 4.373333333333333: Mejor puntuación de la generación: 4
Generación  103: Puntuación media 4.17: Mejor puntuación de la generación: 4
Generación  104: Puntuación media 4.27: Mejor puntuación de la generación: 4
Generación  105: Puntuación media 4.323333333333333: Mejor pu

Generación  190: Puntuación media 4.1866666666666665: Mejor puntuación de la generación: 4
Generación  191: Puntuación media 4.303333333333334: Mejor puntuación de la generación: 4
Generación  192: Puntuación media 4.29: Mejor puntuación de la generación: 4
Generación  193: Puntuación media 4.206666666666667: Mejor puntuación de la generación: 4
Generación  194: Puntuación media 4.283333333333333: Mejor puntuación de la generación: 4
Generación  195: Puntuación media 4.266666666666667: Mejor puntuación de la generación: 4
Generación  196: Puntuación media 4.283333333333333: Mejor puntuación de la generación: 4
Generación  197: Puntuación media 4.176666666666667: Mejor puntuación de la generación: 4
Generación  198: Puntuación media 4.276666666666666: Mejor puntuación de la generación: 4
Generación  199: Puntuación media 4.24: Mejor puntuación de la generación: 4
Generación  200: Puntuación media 4.253333333333333: Mejor puntuación de la generación: 4
Generación  201: Puntuación media 4

Generación  286: Puntuación media 4.3: Mejor puntuación de la generación: 4
Generación  287: Puntuación media 4.256666666666667: Mejor puntuación de la generación: 4
Generación  288: Puntuación media 4.203333333333333: Mejor puntuación de la generación: 4
Generación  289: Puntuación media 4.18: Mejor puntuación de la generación: 4
Generación  290: Puntuación media 4.326666666666667: Mejor puntuación de la generación: 4
Generación  291: Puntuación media 4.296666666666667: Mejor puntuación de la generación: 4
Generación  292: Puntuación media 4.23: Mejor puntuación de la generación: 4
Generación  293: Puntuación media 4.173333333333333: Mejor puntuación de la generación: 4
Generación  294: Puntuación media 4.306666666666667: Mejor puntuación de la generación: 4
Generación  295: Puntuación media 4.27: Mejor puntuación de la generación: 4
Generación  296: Puntuación media 4.23: Mejor puntuación de la generación: 4
Generación  297: Puntuación media 4.256666666666667: Mejor puntuación de la 

Generación  382: Puntuación media 4.323333333333333: Mejor puntuación de la generación: 4
Generación  383: Puntuación media 4.256666666666667: Mejor puntuación de la generación: 4
Generación  384: Puntuación media 4.32: Mejor puntuación de la generación: 4
Generación  385: Puntuación media 4.196666666666666: Mejor puntuación de la generación: 4
Generación  386: Puntuación media 4.28: Mejor puntuación de la generación: 4
Generación  387: Puntuación media 4.17: Mejor puntuación de la generación: 4
Generación  388: Puntuación media 4.28: Mejor puntuación de la generación: 4
Generación  389: Puntuación media 4.206666666666667: Mejor puntuación de la generación: 4
Generación  390: Puntuación media 4.273333333333333: Mejor puntuación de la generación: 4
Generación  391: Puntuación media 4.27: Mejor puntuación de la generación: 4
Generación  392: Puntuación media 4.18: Mejor puntuación de la generación: 4
Generación  393: Puntuación media 4.226666666666667: Mejor puntuación de la generación: 

...

Generación  574: Puntuación media 4.286666666666667: Mejor puntuación de la generación: 4
Generación  575: Puntuación media 4.193333333333333: Mejor puntuación de la generación: 4
Generación  576: Puntuación media 4.27: Mejor puntuación de la generación: 4
Generación  577: Puntuación media 4.286666666666667: Mejor puntuación de la generación: 4
Generación  578: Puntuación media 4.136666666666667: Mejor puntuación de la generación: 4
Generación  579: Puntuación media 4.196666666666666: Mejor puntuación de la generación: 4
Generación  580: Puntuación media 4.156666666666666: Mejor puntuación de la generación: 4
Generación  581: Puntuación media 4.17: Mejor puntuación de la generación: 4
Generación  582: Puntuación media 4.246666666666667: Mejor puntuación de la generación: 4
Generación  583: Puntuación media 4.296666666666667: Mejor puntuación de la generación: 4
Generación  584: Puntuación media 4.216666666666667: Mejor puntuación de la generación: 4
Generación  585: Puntuación media 4.

Generación  670: Puntuación media 4.23: Mejor puntuación de la generación: 4
Generación  671: Puntuación media 4.1866666666666665: Mejor puntuación de la generación: 4
Generación  672: Puntuación media 4.223333333333334: Mejor puntuación de la generación: 4
Generación  673: Puntuación media 4.2: Mejor puntuación de la generación: 4
Generación  674: Puntuación media 4.223333333333334: Mejor puntuación de la generación: 4
Generación  675: Puntuación media 4.27: Mejor puntuación de la generación: 4
Generación  676: Puntuación media 4.156666666666666: Mejor puntuación de la generación: 4
Generación  677: Puntuación media 4.21: Mejor puntuación de la generación: 4
Generación  678: Puntuación media 4.253333333333333: Mejor puntuación de la generación: 4
Generación  679: Puntuación media 4.253333333333333: Mejor puntuación de la generación: 4
Generación  680: Puntuación media 4.173333333333333: Mejor puntuación de la generación: 4
Generación  681: Puntuación media 4.193333333333333: Mejor pun

Generación  766: Puntuación media 4.173333333333333: Mejor puntuación de la generación: 4
Generación  767: Puntuación media 4.166666666666667: Mejor puntuación de la generación: 4
Generación  768: Puntuación media 4.266666666666667: Mejor puntuación de la generación: 4
Generación  769: Puntuación media 4.2: Mejor puntuación de la generación: 4
Generación  770: Puntuación media 4.266666666666667: Mejor puntuación de la generación: 4
Generación  771: Puntuación media 4.203333333333333: Mejor puntuación de la generación: 4
Generación  772: Puntuación media 4.226666666666667: Mejor puntuación de la generación: 4
Generación  773: Puntuación media 4.183333333333334: Mejor puntuación de la generación: 4
Generación  774: Puntuación media 4.18: Mejor puntuación de la generación: 4
Generación  775: Puntuación media 4.183333333333334: Mejor puntuación de la generación: 4
Generación  776: Puntuación media 4.226666666666667: Mejor puntuación de la generación: 4
Generación  777: Puntuación media 4.2

Generación  861: Puntuación media 4.173333333333333: Mejor puntuación de la generación: 4
Generación  862: Puntuación media 4.243333333333333: Mejor puntuación de la generación: 4
Generación  863: Puntuación media 4.226666666666667: Mejor puntuación de la generación: 4
Generación  864: Puntuación media 4.1433333333333335: Mejor puntuación de la generación: 4
Generación  865: Puntuación media 4.196666666666666: Mejor puntuación de la generación: 4
Generación  866: Puntuación media 4.22: Mejor puntuación de la generación: 4
Generación  867: Puntuación media 4.24: Mejor puntuación de la generación: 4
Generación  868: Puntuación media 4.246666666666667: Mejor puntuación de la generación: 4
Generación  869: Puntuación media 4.25: Mejor puntuación de la generación: 4
Generación  870: Puntuación media 4.31: Mejor puntuación de la generación: 4
Generación  871: Puntuación media 4.236666666666666: Mejor puntuación de la generación: 4
Generación  872: Puntuación media 4.223333333333334: Mejor pu

Generación  957: Puntuación media 4.213333333333333: Mejor puntuación de la generación: 4
Generación  958: Puntuación media 4.226666666666667: Mejor puntuación de la generación: 4
Generación  959: Puntuación media 4.24: Mejor puntuación de la generación: 4
Generación  960: Puntuación media 4.173333333333333: Mejor puntuación de la generación: 4
Generación  961: Puntuación media 4.166666666666667: Mejor puntuación de la generación: 4
Generación  962: Puntuación media 4.22: Mejor puntuación de la generación: 4
Generación  963: Puntuación media 4.326666666666667: Mejor puntuación de la generación: 4
Generación  964: Puntuación media 4.24: Mejor puntuación de la generación: 4
Generación  965: Puntuación media 4.273333333333333: Mejor puntuación de la generación: 4
Generación  966: Puntuación media 4.283333333333333: Mejor puntuación de la generación: 4
Generación  967: Puntuación media 4.256666666666667: Mejor puntuación de la generación: 4
Generación  968: Puntuación media 4.2333333333333

El problema anterior es mucho más complejo, vemos como al principio aprende rápido pero pronto se queda estancada. Este fenómeno lo hemos estudiado en clase, pues el aprendizaje de este tipo de algoritmos comienza siendo muy rápido pero a medida que aumentan las generaciones tiende a estancarse.